# Получение кадров с трансляции

In [ ]:
import os
import cv2
import time

from ultralytics import YOLO

In [ ]:
# capture_frames.py


RTSP_URL = "https://restreamer.vms.evo73.ru/918335436b92ac26/stream.m3u8"  # подставь свой поток
OUT_DIR = "dataset/images"
os.makedirs(OUT_DIR, exist_ok=True)

cap = cv2.VideoCapture(RTSP_URL)
fps = cap.get(cv2.CAP_PROP_FPS) or 25
grab_every = int(fps)  # один кадр в секунду (при fps ~1 кадр/sec, уменьшить)

i = 0
saved = 0
while True:
    ret, frame = cap.read()
    if not ret:
        time.sleep(0.5)
        continue
    if i % grab_every == 0:
        fname = f"{saved:06d}.jpg"
        cv2.imwrite(os.path.join(OUT_DIR, fname), frame)
        saved += 1
        print("Saved", fname)
    i += 1
    # остановка вручную:
    if saved >= 500:  # например 500 кадров
        break

cap.release()


# Работа с Yolo и разметка кадров

In [ ]:
from ultralytics import YOLO
import os, json

model = YOLO('yolov10n.pt')  # предобученная
img_dir = "dataset/images"
out_ann_dir = "dataset/labels" 
os.makedirs(out_ann_dir, exist_ok=True)

for img in os.listdir(img_dir):
    if not img.endswith('.jpg'): continue
    res = model.predict(os.path.join(img_dir,img), imgsz=640, conf=0.3)
    # res[0].boxes.xyxyn, res[0].boxes.cls, res[0].masks ... можно парсить и сохранять YOLO txt
    boxes = res[0].boxes
    h, w = res[0].orig_shape
    lines = []
    for b in boxes:
        x1,y1,x2,y2 = b.xyxy[0].tolist()
        cls = int(b.cls[0].item())
        # convert to yolo format
        cx = ((x1 + x2) / 2) / w
        cy = ((y1 + y2) / 2) / h
        bw = (x2 - x1) / w
        bh = (y2 - y1) / h
        lines.append(f"{cls} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")
    with open(os.path.join(out_ann_dir, img.replace('.jpg','.txt')), 'w') as f:
        f.write("\n".join(lines))
